# Monitoring a Model

When you've deployed a model into production as a service, you'll want to monitor it to track usage and explore the requests it processes. In this lab, you'll use Azure Application Insights to monitor activity for a model service endpoint.

## Before You Start

Before you start this lab, ensure that you have completed the *Create an Azure Machine Learning Workspace* and *Create a Compute Instance* tasks in [Lab 1: Getting Started with Azure Machine Learning](./labdocs/Lab01.md). Then open this notebook in Jupyter on your Compute Instance.

## Connect to Your Workspace

The first thing you need to do is to connect to your workspace using the Azure ML SDK.

> **Note**: You may be prompted to authenticate. Just copy the code and click the link provided to sign into your Azure subscription, and then return to this notebook.

In [15]:
import json
import os
import joblib as joblib
import requests
from azureml.core import Workspace, Dataset, Experiment
import pandas as pd
import numpy as np
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice
from sklearn import model_selection, metrics
from sklearn.tree import DecisionTreeClassifier


In [2]:
ws = Workspace.from_config()
print('Ready to work with', ws.name)


Ready to work with workspace


## Prepare a Model for Deployment

Now we need a model to deploy. Run the code below to:

1. Create and register a dataset.
2. Train a model using the dataset.
3. Register the model.

In [4]:
default_ds = ws.get_default_datastore()
default_ds.upload_files(
    ['./data/diabetes.csv', './data/diabetes2.csv'],
    target_path='diabetes-data/',
    overwrite=True,
)

print('Creating dataset...')
data_set = Dataset.Tabular.from_delimited_files(
    (default_ds, 'diabetes-data/*.csv')
)

print('Registering dataset...')
data_set = data_set.register(
    ws, 'diabetes dataset', description='diabetes data',
    tags = {'format':'CSV'}, create_new_version=True
)

experiment = Experiment(ws, 'diabetes-training')
run = experiment.start_logging()
print('Starting experiment:', experiment.name)

print('Loading Data...')
diabetes: pd.DataFrame = data_set.to_pandas_dataframe()

X = diabetes[
        [
            'Pregnancies', 'PlasmaGlucose', 'DiastolicBloodPressure',
            'TricepsThickness', 'SerumInsulin', 'BMI', 'DiabetesPedigree',
            'Age',
        ]
    ].to_numpy()
y = diabetes['Diabetic'].to_numpy()

X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, y, test_size=0.30, random_state=0
)

print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', acc)

y_scores = model.predict_proba(X_test)
auc = metrics.roc_auc_score(y_test,y_scores[:,1])
print('AUC:', auc)
run.log('AUC', auc)

model_file = 'diabetes_model.pkl'
joblib.dump(model, model_file)
run.upload_file('outputs/' + model_file, './' + model_file)

run.complete()

print('Registering model...')
run.register_model(
    'diabetes_model', model_path='outputs/diabetes_model.pkl',
    tags={'Training context':'Inline Training'},
    properties={
        'AUC': run.get_metrics()['AUC'], 
        'Accuracy': run.get_metrics()['Accuracy']
    }
)

model = ws.models['diabetes_model']

print('Model trained and registered.')


Uploading an estimated of 2 files
Uploading ./data/diabetes.csv
Uploading ./data/diabetes2.csv
Uploaded ./data/diabetes.csv, 1 files out of an estimated total of 2
Uploaded ./data/diabetes2.csv, 2 files out of an estimated total of 2
Uploaded 2 files
Creating dataset...
Registering dataset...
Starting experiment: diabetes-training
Loading Data...
Training a decision tree model
Accuracy: 0.8975555555555556
AUC: 0.8827569166900401
Registering model...
Model trained and registered.


## Deploy a Model as a Web Service

Now you're ready to deploy the registered model as a web service.

First, create a folder for the deployment configuration files

In [6]:
folder_name = 'diabetes-service'
os.makedirs(folder_name, exist_ok=True)
print(folder_name)


diabetes-service


Now you need an entry script that the service will use to score new data.

In [7]:
%%writefile $folder_name/score_diabetes.py
import json
import joblib
import numpy as np
from azureml.core.model import Model
from sklearn.linear_model import LogisticRegression

model: LogisticRegression


def init():
    """
    Initialize `model`
    """
    global model
    model_path = Model.get_model_path('diabetes_model')
    model = joblib.load(model_path)


def run(raw_data):
    """
    Score `model`
    """
    data = json.loads(raw_data)['data']
    np_data = np.array(data)
    predictions = model.predict(np_data)
    log_text = f'Data: {data} - Predictions: {predictions}'
    print(log_text)
    classnames = ['not-diabetic', 'diabetic']
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
    return json.dumps(predicted_classes)


Overwriting diabetes-service/score_diabetes.py


You'll also need a Conda configuration file for the service environment.

In [9]:
myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")

env_file = folder_name + "/diabetes_env.yml"
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)


Saved dependency info in diabetes-service/diabetes_env.yml


Now you can deploy the service (in this case, as an Azure Container Instance (ACI).

> **Note**: This can take a few minutes - wait until the state is shown as **Healthy**.

In [11]:
inference_config = InferenceConfig(
    f'score_diabetes.py', runtime='python', conda_file='diabetes_env.yml',
    source_directory = folder_name,
)

service_name = "diabetes-service-app-insights"
deployment_config = AciWebservice.deploy_configuration(
    cpu_cores = 1, memory_gb = 1
)
aci_service = Model.deploy(
    ws, service_name, [model],
    inference_config= inference_config, deployment_config=deployment_config
)
aci_service.wait_for_deployment(show_output = True)
print(aci_service.state)


Running......................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


## Enable Application Insights

Next, you need to enable Application Insights for the service.

In [13]:
aci_service.update(enable_app_insights=True)
print(aci_service.state)
print('AppInsights enabled!')


Healthy
AppInsights enabled!


## Use the Web Service

With the service deployed, now you can consume it from a client application.

First, determine the URL to which these applications must submit their requests.

In [14]:
print(aci_service.state)
endpoint = aci_service.scoring_uri
print(endpoint)


Healthy
http://d5c06dcd-2e65-408f-806b-8d1841fc82ad.westus.azurecontainer.io/score


Now that you know the endpoint URI, an application can simply make an HTTP request, sending the patient data in JSON (or binary) format, and receive back the predicted class(es).

> **Tip**: If an error occurs because the service endpoint isn't ready. Wait a few seconds and try again!

In [16]:
x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

input_json = json.dumps({"data": x_new})

headers = { 'Content-Type':'application/json' }

predictions = requests.post(endpoint, input_json, headers = headers)
print(predictions.status_code)
if predictions.status_code == 200:
    predicted_classes = json.loads(predictions.json())
    for i in range(len(x_new)):
        print (f'Patient {x_new[i]}', predicted_classes[i] )


200
Patient [2, 180, 74, 24, 21, 23.9091702, 1.488172308, 22] diabetic
Patient [0, 148, 58, 11, 179, 39.19207553, 0.160829008, 45] not-diabetic


Now you can view the data logged for the service endpoint:
1. In the [Azure portal](https://portal.azure.com), open your Machine Learning workspace.
2. On the **Overview** page, click the link for the associated **Application Insights** resource.
3. On the Application Insights blade, click **Logs**. 

    > **Note**: If this is the first time you've opened log analytics, you may need to click **Get Started** to open the query editor. If a tip explaining how to write a query is displayed, close it.

4. Paste the following query into the query editor and click **Run**
    ```
    traces
    |where  message == "STDOUT"
      and customDimensions.["Service Name"] == "diabetes-service-app-insights"
    |project timestamp, customDimensions.Content
    ```
5. View the results. At first there may be none, because an ACI web service can take two to three minutes to send the telemetry to Application Insights. Wait a few minutes and re-run the query until you see the logged data and predictions.

## Delete the Service

When you no longer need your service, you should delete it to avoid incurring unecessary charges.

In [ ]:
aci_service.delete()
print('Service deleted.')


For more information about using Application Insights to monitor a deployed service, see the [Azure Machine Learning documentation](https://docs.microsoft.com/azure/machine-learning/how-to-enable-app-insights).